# Fixations

In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))
from config import *
from features.feature_utils import *

In [3]:
experiment = "FIXATIONS"
participant_ids = pd.read_parquet(
        f"{PREPROCESSED_DIR}/{experiment}_samples.pq", 
        columns=["participant_id"]
    )

participant_ids = participant_ids["participant_id"].unique()

In [4]:
def get_samples_df(participant_id, experiment):
    df_samples = pd.read_parquet(
        f"{PREPROCESSED_DIR}/{experiment}_samples.pq",
        filters=[('participant_id', '=', participant_id)]
        )
    return df_samples
    
def get_events_df(participant_id, experiment):
    df_events = pd.read_parquet(
        f"{PREPROCESSED_DIR}/{experiment}_events.pq",
        filters=[('participant_id', '=', participant_id)]
        )
    
    return df_events

In [6]:
participant_id = participant_ids[0]

df_sample = get_samples_df(participant_id, experiment)
df_event = get_events_df(participant_id, experiment)
df_combined = combine_samples_events(df_sample, df_event, experiment)

In [7]:
get_distance_to_stimulus_features(df_combined)

,experiment,participant_id,distance_to_fixpoint_mean,distance_to_fixpoint_min,distance_to_fixpoint_max,distance_to_fixpoint_median,distance_to_fixpoint_std,distance_to_fixpoint_x_mean,distance_to_fixpoint_x_min,distance_to_fixpoint_x_max,distance_to_fixpoint_x_median,distance_to_fixpoint_x_std,distance_to_fixpoint_y_mean,distance_to_fixpoint_y_min,distance_to_fixpoint_y_max,distance_to_fixpoint_y_median,distance_to_fixpoint_y_std
0,FIXATIONS,106,56.324557,6.363988,132.128261,45.086285,31.72709,1388.252463,0.02,7893.86,1452.525,1146.094091,2823.042479,0.01,13844.945,842.85,3814.696839


In [ ]:
(
    df_event.sort_values(by=["participant_id", "trial_id", "time"])
        .groupby(["participant_id", "trial_id"])["time"]
        .transform(lambda group: (
            group.iloc[-1]
        ))
)

In [25]:
(df_event.query("stimulus_active == True")
    .sort_values(by=["participant_id", "trial_id","time"])
    .assign(stimulus_time = lambda x: np.select([x.event == "FIXPOINT", x.event != "FIXPOINT"], [x.time, None]))
    .assign(stimulus_time = lambda x: x["stimulus_time"].ffill())
    .assign(max_event_time = lambda x: (
            x.sort_values(by=["participant_id", "trial_id", "time"])
            .groupby(["participant_id", "trial_id"])["time"]
            .transform(lambda group: (
                group.iloc[-1]
            ))
        ))
    .assign(trial_active_duration_seconds = lambda x: (x["max_event_time"] - x["stimulus_time"])/1000)
    .query("event == 'ESACC'")
    .groupby(["experiment", "participant_id", "trial_id", "eye", "trial_active_duration_seconds"])
    .size()
    .reset_index(name="n_saccades")
    .assign(
        saccades_per_second_raw = lambda x: x["n_saccades"] / x["trial_active_duration_seconds"]
    )
    .groupby(["experiment", "participant_id", "trial_id"])
    .agg(
        total_saccades_per_second = ("saccades_per_second_raw", "sum"),
        n_eyes = ("eye", "nunique")
    )
    .reset_index()
    .assign(
        saccades_per_second = lambda x: x["total_saccades_per_second"] / x["n_eyes"]
    )
    .groupby(["experiment", "participant_id"])
    .agg({
        'saccades_per_second': [np.mean, np.min, np.max, np.median, np.std],
    })
    
    .reset_index()
    .pipe(rename_columns)
)

,experiment,participant_id,saccades_per_second_mean,saccades_per_second_min,saccades_per_second_max,saccades_per_second_median,saccades_per_second_std
0,FIXATIONS,106,0.322727,0.19861,0.59579,0.248254,0.184037
